In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC

In [ ]:
from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, Lambda, LSTM, TimeDistributed, Masking, Bidirectional
from keras.layers import Reshape, Flatten, Dropout, Concatenate, Activation, MaxPool2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Model, load_model, Sequential
from keras.optimizers import Adam
import keras.backend as K

In [22]:
import gzip
from gensim.models import Word2Vec

In [58]:
dev_raw = pd.read_csv(os.path.join(os.getcwd(), 'dev_sent_emo.csv'))
train_raw = pd.read_csv(os.path.join(os.getcwd(), 'train_sent_emo.csv'))
test_raw = pd.read_csv(os.path.join(os.getcwd(), 'test_sent_emo.csv'))

In [59]:
dev_raw.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"


In [60]:
dev_raw.Utterance = dev_raw.Utterance.apply(lambda x: re.sub('\\x92', "'", x))

In [61]:
dev_raw

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, he's lost it. He's totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,You're a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we won't be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"
5,6,"Now, there's two reasons.",Chandler,neutral,neutral,1,3,4,4,"00:12:38,466","00:12:39,841"
6,7,Hey.,Phoebe,neutral,neutral,1,4,4,4,"0:12:43,012","0:12:44,074"
7,8,Hey!,All,joy,positive,1,5,4,4,"0:12:44,074","0:12:45,189"
8,9,"Ohh, you guys, remember that cute client I tol...",Phoebe,neutral,neutral,1,6,4,4,"00:12:45,347","00:12:50,852"
9,10,Where?!,Rachel,surprise,negative,1,7,4,4,"0:12:52,730","0:12:55,002"


In [ ]:
model = Word2Vec(sentences, size=10, window=5, min_count=5, workers=1)

In [39]:
model = gensim.models.Word2Vec(
    docs,
    size=150,
    window=10,
    min_count=2,
    workers=10)

In [41]:
model.train(docs, total_examples=len(docs), epochs=10)

(100073, 446460)

In [46]:
model.vocabulary.cum_table

array([ 248996261,  390183681,  516383477,  636280609,  741849280,
        835592081,  926991496, 1012132433, 1093423038, 1163615804,
       1227621693, 1291589720, 1352195751, 1408904045, 1455609138,
       1502272174, 1548133689, 1593783503, 1638668460, 1674314607,
       1709036440, 1743386250, 1776375995, 1805952997, 1833848937,
       1861494838, 1885555043, 1907916312, 1928699890, 1946199057,
       1963581434, 1979780968, 1994211835, 2004582199, 2014321065,
       2023486925, 2032288540, 2040869185, 2049226946, 2056903679,
       2064426360, 2071559195, 2078613214, 2085348963, 2091842688,
       2097076160, 2102045715, 2107015269, 2111716147, 2115863452,
       2119820777, 2123388600, 2126129832, 2128871063, 2131503360,
       2134025198, 2136434939, 2138730818, 2140424690, 2141857531,
       2143153826, 2144450122, 2145457304, 2145963890, 2146470475,
       2146977061, 2147483647], dtype=uint32)

In [44]:
w1 = ['God']
model.wv.most_similar(positive=w1, topn=6)

KeyError: "word 'God' not in vocabulary"